## Agreements

In [1]:
import os
import sys
import django
from tqdm.auto import tqdm

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

sys.path.append("..")
os.environ['DJANGO_SETTINGS_MODULE'] = 'news_labelling.settings'
django.setup()

from api.models import Comment, ArticleLabel, CommentLabel, Article
from django.contrib.auth.models import User

Primero, veamos qué usuarios etiquetaron más de 4 artículos...

In [2]:
from django.db.models import Count

users = []
for agg in ArticleLabel.objects.values('user__username').annotate(Count('user__username')):
    if agg['user__username__count'] >= 9:
         users.append(agg['user__username'])   
users

['jmperez', 'jzajac', 'mkondra', 'ndebandi']

In [3]:
base_query = {
    "user__username__in": users
}
ArticleLabel.objects.filter(**base_query).count()

83

¿Cuántos etiquetamos todes?

In [4]:
import pandas as pd

pd.options.display.max_columns = 40


articles_labelled = {label.article_id for label in ArticleLabel.objects.filter(**base_query)}

df = pd.DataFrame({"name":users})
df.set_index("name", inplace=True)

for label in ArticleLabel.objects.filter(**base_query):
    username = label.user.username
    df.loc[username, label.article.title] = int(label.is_interesting)
    
df

,Condenan a ocho años de cárcel por corrupción al ex presidente ecuatoriano Rafael Correa,"Cecilia Roth: ""La marcha anticuarentena me pareció una irresponsabilidad sanitaria""",El primer muerto en los motines de las cárceles de Santa Fe fue un barra de Colón,El proyecto del impuesto a la riqueza “está listo” y se presentará la próxima semana,Ante el cierre de los cines por la pandemia el presidente del INCAA quiere ponerle otro impuesto a Netflix,Cecilia Roth: “La marcha anticuarentena me pareció una irresponsabilidad sanitaria”,"Sergio Berni: “Debemos ir a una cuarentena absoluta, mucho más rigurosa que al principio y sin transporte público”",Los piratas del siglo XXI se roban US$1000 millones por año del mar argentino,Juntos por el Cambio impulsa un proyecto en la provincia de Buenos Aires para sacarles los planes sociales a los que usurpen terrenos,¿Terminará Argentina como Venezuela?,"Al borde de las lágrimas, Luis Brandoni convocó un banderazo en contra del Gobierno, pero “cumpliendo todos los protocolos”",Coronavirus en la Argentina: Hebe de Bonafini pidió suspender la marcha del 24 de marzo,Intentó violar a su novia y ella lo asesinó a escobazos,Ginés González García admite que prohibieron a los runners por una cuestión de imagen “más que por los contagios”,"Solo el 35,5% de la población aprueba la gestión de la pandemia realizada por el Gobierno",Verónica Magario pidió endurecer la cuarentena: “Debemos frenar la circulación de personas en el AMBA para evitar contagios”,"Según un experto italiano, “el coronavirus se está debilitando y podría morir solo sin una vacuna”",Video: Jair Bolsonaro le hace upa a un hombre de talla baja pensando que era un niño,"Filminas para todos, menos para Maduro",Ginés González: “La semana que viene aviones de Aerolíneas irán a China a buscar insumos de bioseguridad”,Florencia Peña se diferenció de los argentinos que se quieren ir del país: “Muchos queremos seguir apostando”,Así fue el minuto a minuto del debate entre los candidatos a vicepresidente de Estados Unidos,Escándalo en España: fallan los tests de coronavirus fabricados en China,"""Evolucionen"": Romina Malaspina hizo un fuerte descargo tras los comentarios sexistas por su look en el noticiero","Luis Novaresio contó cómo será su boda con Braulio Bauab: ""Nos va a casar una rabina"""
name,,,,,,,,,,,,,,,,,,,,,,,,,
jmperez,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0
jzajac,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
mkondra,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN
ndebandi,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Busquemos los que están etiquetados por todos

In [7]:
df = df[df.columns[df.notna().all()]]

df

,Condenan a ocho años de cárcel por corrupción al ex presidente ecuatoriano Rafael Correa,"Cecilia Roth: ""La marcha anticuarentena me pareció una irresponsabilidad sanitaria""",El primer muerto en los motines de las cárceles de Santa Fe fue un barra de Colón,El proyecto del impuesto a la riqueza “está listo” y se presentará la próxima semana,Ante el cierre de los cines por la pandemia el presidente del INCAA quiere ponerle otro impuesto a Netflix,Cecilia Roth: “La marcha anticuarentena me pareció una irresponsabilidad sanitaria”,"Sergio Berni: “Debemos ir a una cuarentena absoluta, mucho más rigurosa que al principio y sin transporte público”",Los piratas del siglo XXI se roban US$1000 millones por año del mar argentino,Juntos por el Cambio impulsa un proyecto en la provincia de Buenos Aires para sacarles los planes sociales a los que usurpen terrenos,¿Terminará Argentina como Venezuela?,"Al borde de las lágrimas, Luis Brandoni convocó un banderazo en contra del Gobierno, pero “cumpliendo todos los protocolos”",Coronavirus en la Argentina: Hebe de Bonafini pidió suspender la marcha del 24 de marzo,Intentó violar a su novia y ella lo asesinó a escobazos
name,,,,,,,,,,,,,
jmperez,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0
jzajac,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
mkondra,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
ndebandi,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [8]:
from nltk import agreement

coders = df.values

formatted_labels = []
for i in range(coders.shape[0]):
    # Acá las ponemos siguiendo el siguiente formato
    # [num etiquetador, num instancia, label]
    for j in range(coders.shape[1]):
        formatted_labels.append((i, j, coders[i, j]))
        

ratingtask = agreement.AnnotationTask(data=formatted_labels)

ratingtask.kappa()

0.22573839662447262

Ergh...bueno

## Agreements sobre odio

Ok, acá agarremos todos los labels sobre comentarios de todes

In [9]:
users = [
    "jmperez", 
    "mkondra", 
    "ndebandi",
    "jzajac",
]

article_labels = ArticleLabel.objects.filter(
    user__username__in=users,
)

formatted_labels = []

df_comments = pd.DataFrame({"name": users})

df_comments.set_index("name", inplace=True)

for article_label in article_labels:
    username = article_label.user.username
    for comment_label in article_label.comment_labels.all():
        df_comments.loc[username, comment_label.comment_id] = comment_label.is_hateful

# Saco los que no estén etiquetados por todes
#df_comments = df_comments[df_comments.columns[df_comments.notna().all()]]

df_comments

,294781,294782,294783,294784,294785,294786,294787,294788,294789,294790,294791,294792,294793,294794,294795,294796,294797,294798,294799,294800,...,311844,311845,311846,311847,311848,311849,311850,311851,311852,311853,311854,311855,311856,311857,311858,311859,311860,311861,311862,311863
name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
jmperez,False,False,False,False,False,False,False,False,False,True,False,False,True,True,False,True,True,True,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mkondra,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
ndebandi,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
jzajac,False,True,False,False,False,False,False,False,True,True,True,False,True,True,True,True,True,True,False,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
from nltk import agreement

def kappa_of(subset):
    coders = df_comments.loc[subset]
    
    coders = coders[coders.columns[coders.notna().all()]]
    
    coders = coders.values
    formatted_labels = []
    for i in range(coders.shape[0]):
        # Acá las ponemos siguiendo el siguiente formato
        # [num etiquetador, num instancia, label]
        for j in range(coders.shape[1]):
            formatted_labels.append((i, j, coders[i, j]))


    ratingtask = agreement.AnnotationTask(data=formatted_labels)

    return ratingtask.kappa(), coders.shape[1]


for subset in [
    ["jmperez", "mkondra", "ndebandi", "jzajac"],
    ["jmperez", "mkondra", "ndebandi"],
    ["jmperez", "mkondra", "jzajac"],
    ["mkondra", "ndebandi", "jzajac"],
    ]:
    name = ' '.join(subset)
    kappa, instances = kappa_of(subset)
    print(f"Fleiss K para {name:<40} = {kappa:.3f} (#instancias = {instances})")
    
print("\n\nPares\n\n")

for i in range(len(users)):
    for j in range(i+1, len(users)):
        subset = [users[i], users[j]]
        name = ' '.join(subset)
        kappa, instances = kappa_of(subset)
        print(f"Fleiss K para {name:<40} = {kappa:.3f} (#instancias = {instances})")


Fleiss K para jmperez mkondra ndebandi jzajac          = 0.414 (#instancias = 273)
Fleiss K para jmperez mkondra ndebandi                 = 0.526 (#instancias = 273)
Fleiss K para jmperez mkondra jzajac                   = 0.481 (#instancias = 494)
Fleiss K para mkondra ndebandi jzajac                  = 0.417 (#instancias = 521)


Pares


Fleiss K para jmperez mkondra                          = 0.641 (#instancias = 494)
Fleiss K para jmperez ndebandi                         = 0.438 (#instancias = 273)
Fleiss K para jmperez jzajac                           = 0.443 (#instancias = 688)
Fleiss K para mkondra ndebandi                         = 0.542 (#instancias = 521)
Fleiss K para mkondra jzajac                           = 0.378 (#instancias = 742)
Fleiss K para ndebandi jzajac                          = 0.366 (#instancias = 521)


In [11]:

df_comments.sum(axis=1)

name
jmperez     204.0
mkondra     122.0
ndebandi     96.0
jzajac      481.0
dtype: float64

## Agreements sobre cada categoría

In [44]:
users = [
    "jmperez", 
    "mkondra", 
    #"ndebandi",
    "jzajac",
]


idx = pd.MultiIndex.from_product([CommentLabel.type_mapping, users],
                                names=["categoria", "etiquetador"])

df_labels = pd.DataFrame(index=idx)

df_labels

Empty DataFrame
Columns: []
Index: [(MUJER, jmperez), (MUJER, mkondra), (MUJER, jzajac), (LGBTI, jmperez), (LGBTI, mkondra), (LGBTI, jzajac), (RACISMO, jmperez), (RACISMO, mkondra), (RACISMO, jzajac), (POBREZA, jmperez), (POBREZA, mkondra), (POBREZA, jzajac), (POLITICA, jmperez), (POLITICA, mkondra), (POLITICA, jzajac), (DISCAPACIDAD, jmperez), (DISCAPACIDAD, mkondra), (DISCAPACIDAD, jzajac), (ASPECTO, jmperez), (ASPECTO, mkondra), (ASPECTO, jzajac), (CRIMINAL, jmperez), (CRIMINAL, mkondra), (CRIMINAL, jzajac), (OTROS, jmperez), (OTROS, mkondra), (OTROS, jzajac)]

In [45]:

article_labels = ArticleLabel.objects.filter(
    user__username__in=users,
)


for article_label in article_labels:
    username = article_label.user.username
    for comment_label in article_label.comment_labels.all():
        for name, field_name in CommentLabel.type_mapping.items():
            df_labels.loc[(name, username), comment_label.comment_id] = getattr(comment_label, field_name)

df_labels

294781 294782 294783 294784 294785 294786 294787  \
categoria    etiquetador                                                    
MUJER        jmperez      False  False  False  False  False  False  False   
             mkondra      False  False  False  False  False  False  False   
             jzajac       False  False  False  False  False  False  False   
LGBTI        jmperez      False  False  False  False  False  False  False   
             mkondra      False  False  False  False  False  False  False   
             jzajac       False  False  False  False  False  False  False   
RACISMO      jmperez      False  False  False  False  False  False  False   
             mkondra      False  False  False  False  False  False  False   
             jzajac       False  False  False  False  False  False  False   
POBREZA      jmperez      False  False  False  False  False  False  False   
             mkondra      False  False  False  False  False  False  False   
             jzajac       False  False  False  False  False  False  False   
POLITICA     jmperez      False  False  False  False  False  False  False   
             mkondra      False  False  False  False  False  False  False   
             jzajac       False   True  False  False  False  False  False   
DISCAPACIDAD jmperez      False  False  False  False  False  False  False   
             mkondra      False  False  False  False  False  False  False   
             jzajac       False  False  False  False  False  False  False   
ASPECTO      jmperez      False  False  False  False  False  False  False   
             mkondra      False  False  False  False  False  False  False   
             jzajac       False  False  False  False  False  False  False   
CRIMINAL     jmperez      False  False  False  False  False  False  False   
             mkondra      False  False  False  False  False  False  False   
             jzajac       False  False  False  False  False  False  False   
OTROS        jmperez      False  False  False  False  False  False  False   
             mkondra      False  False  False  False  False  False  False   
             jzajac       False  False  False  False  False  False  False   

                         294788 294789 294790 294791 294792 294793 294794  \
categoria    etiquetador                                                    
MUJER        jmperez      False  False  False  False  False  False  False   
             mkondra      False  False  False  False  False  False  False   
             jzajac       False  False  False   True  False  False   True   
LGBTI        jmperez      False  False  False  False  False  False  False   
             mkondra      False  False  False  False  False  False  False   
             jzajac       False  False  False  False  False  False  False   
RACISMO      jmperez      False  False  False  False  False  False  False   
             mkondra      False  False  False  False  False  False  False   
             jzajac       False  False  False  False  False  False  False   
POBREZA      jmperez      False  False  False  False  False  False  False   
             mkondra      False  False  False  False  False  False  False   
             jzajac       False  False  False  False  False  False  False   
POLITICA     jmperez      False  False   True  False  False   True  False   
             mkondra      False  False  False  False  False  False  False   
             jzajac       False  False   True  False  False   True  False   
DISCAPACIDAD jmperez      False  False  False  False  False  False  False   
             mkondra      False  False  False  False  False  False  False   
             jzajac       False  False  False  False  False  False  False   
ASPECTO      jmperez      False  False  False  False  False  False   True   
             mkondra      False  False  False  False  False  False   True   
             jzajac       False   True  False  False  False  False   True   
CRIMINAL     jmperez

In [46]:

def kappa_of(df):
    
    coders = df[df.columns[df.notna().all()]]
    
    coders = coders.values
    formatted_labels = []
    for i in range(coders.shape[0]):
        # Acá las ponemos siguiendo el siguiente formato
        # [num etiquetador, num instancia, label]
        for j in range(coders.shape[1]):
            formatted_labels.append((i, j, coders[i, j]))


    ratingtask = agreement.AnnotationTask(data=formatted_labels)

    return ratingtask.kappa(), coders.shape[1]


for category in CommentLabel.type_mapping:
    try:
        kappa, instances = kappa_of(df_labels.loc[category])
        print(f"K {category:<12} = {kappa:.3f}")
    except ZeroDivisionError as e:
        print(f"No hay elementos para calcular Kappa de {category} - salteando")

print(f"Instancias: {instances}")
print(users)

K MUJER        = 0.446
No hay elementos para calcular Kappa de LGBTI - salteando
K RACISMO      = 0.362
K POBREZA      = 0.404
K POLITICA     = 0.292
K DISCAPACIDAD = 0.252
K ASPECTO      = 0.661
K CRIMINAL     = 0.722
K OTROS        = -0.002
Instancias: 494
['jmperez', 'mkondra', 'jzajac']


In [49]:
df_labels.columns[df_labels.loc["OTROS"].any()]

Int64Index([294796, 294813, 294822, 294827, 263402, 263406, 262080, 262085,
            262095, 262097, 262105, 262109, 267710, 267724, 267749, 268643,
            297563, 304432, 304437, 304438, 304439, 304443, 304444, 304449,
            304452, 304456, 304457, 304465, 304470, 304471, 304473, 304474,
            304476, 304480, 311883, 320122, 320134, 320136, 311828],
           dtype='int64')

## Error Analysis

In [44]:
df_comments = df_comments.loc[["mkondra", "ndebandi", "jzajac"]]
all_labeled = df_comments[df_comments.columns[df_comments.notna().all()]]


agree = (all_labeled.sum() == 0) | (all_labeled.sum() == all_labeled.shape[0])

no_agree_comments = all_labeled[all_labeled.columns[~agree]]

In [45]:
for comment_id in no_agree_comments.columns:
    comment = Comment.objects.get(id=comment_id)
    
    article = comment.article
    
    print("="*80)
    
    print(article.title)
    print(comment.text)
    
    print(no_agree_comments[comment_id])

Cecilia Roth: "La marcha anticuarentena me pareció una irresponsabilidad sanitaria"
@LANACION Quédate encerrada en tu casa sin trabajar hasta que ( si es que no recibís sobres) te cagues de hambre y te hundas profundamente en una quiebra irreversible
name
mkondra     False
ndebandi    False
jzajac       True
Name: 294782, dtype: object
Cecilia Roth: "La marcha anticuarentena me pareció una irresponsabilidad sanitaria"
@LANACION Te comiste a Feto Paez irresponsable‼
name
mkondra     False
ndebandi    False
jzajac       True
Name: 294789, dtype: object
Cecilia Roth: "La marcha anticuarentena me pareció una irresponsabilidad sanitaria"
@LANACION Estas aparecen cuando están cagadas de hambre y les pasan el sobre!!!!😬😡
name
mkondra     False
ndebandi    False
jzajac       True
Name: 294790, dtype: object
Cecilia Roth: "La marcha anticuarentena me pareció una irresponsabilidad sanitaria"
@LANACION Que hace esta mina o sra no se
name
mkondra     False
ndebandi    False
jzajac       True
Name: